In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
from torch import nn, optim
import matplotlib.pyplot as plt

from itertools import product

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Step 1: Generate the true class balance to be recovered

In [2]:
K = 3

# Generate the true class balance to be recovered
p_Y = np.random.random(K)
p_Y /= p_Y.sum()
p_Y

array([0.56401127, 0.29391442, 0.1420743 ])

### Step 2: Generate the true conditional probability tables (CPTs) for the LFs

Separate simple process here to keep simple (later merge this with the SPA generator).
Generate in terms of the _conditional accuracies_ (which is equivalent to the recall...):
$$
\alpha_{i,y',y} = P(\lambda_i = y' | Y = y)
$$

Note that this table should be normalized such that:
$$
\sum_{y'} \alpha_{i,y',y} = 1
$$

In [3]:
M = 10
alphas = []
for i in range(M):
    a = np.random.random((K,K))
    alphas.append( a @ np.diag(1 / a.sum(axis=0)) )
alpha = np.array(alphas)

assert np.all(np.abs(alpha.sum(axis=1) - 1) < 1e-5)
alpha[0]

array([[0.4754549 , 0.27264056, 0.26943463],
       [0.06054756, 0.18850857, 0.28989351],
       [0.46399755, 0.53885088, 0.44067186]])

### Aside: Different tensor product approaches in Python

#### (1) Brute force

In [4]:
%%time
O_1 = np.zeros((M,M,M,K,K,K))
for i, j, k, y1, y2, y3 in product(range(M), range(M), range(M), range(K), range(K), range(K)):
    for y in range(K):
        O_1[i,j,k,y1,y2,y3] += alpha[i, y1, y] * alpha[j, y2, y] * alpha[k, y3, y]

CPU times: user 116 ms, sys: 1.92 ms, total: 118 ms
Wall time: 118 ms


#### (3) `np.einsum`

In [5]:
%time O_3 = np.einsum('aby,cdy,efy->acebdf', alpha, alpha, alpha)

CPU times: user 1.39 ms, sys: 549 µs, total: 1.94 ms
Wall time: 1.17 ms


In [6]:
np.mean(np.abs(O_1 - O_3))

3.962686660289687e-18

Now, trying a bilinear form:

In [7]:
%%time
Op_1 = np.zeros((M,M,M,K,K,K))
for i, j, k, y1, y2, y3 in product(range(M), range(M), range(M), range(K), range(K), range(K)):
    for y in range(K):
        Op_1[i,j,k,y1,y2,y3] += p_Y[y] * alpha[i, y1, y] * alpha[j, y2, y] * alpha[k, y3, y]

CPU times: user 507 ms, sys: 8.58 ms, total: 516 ms
Wall time: 136 ms


In [8]:
%time Op_3 = np.einsum('aby,cdy,efy,y->acebdf', alpha, alpha, alpha, p_Y)

CPU times: user 736 µs, sys: 138 µs, total: 874 µs
Wall time: 533 µs


In [9]:
np.mean(np.abs(Op_1 - Op_3))

2.043199071866942e-18

### Step 3: Generate the _three-way_ overlaps tensor $O$ of conditionally-independent LFs

Now we can directly generate $O$.
By our conditional independence assumption, we have:
$$
P(\lambda_i = y', \lambda_j = y'' | Y = y) = \alpha_{i,y',y} \alpha_{j,y'',y}
$$

Thus we have:
$$
O_{i,j,y',y''} = \sum_y P(Y=y) \alpha_{i,y',y} \alpha_{j,y'',y}
$$

In [10]:
# Compute mask
mask = torch.ones((M,M,M,K,K,K)).byte()
for i, j, k in product(range(M), repeat=3):
    if len(set((i,j,k))) < 3:
        mask[i,j,k,:,:,:] = 0

In [11]:
%%time
O = np.einsum('aby,cdy,efy,y->acebdf', alpha, alpha, alpha, p_Y)
O = torch.from_numpy(O).float()
O[1-mask] = 0

CPU times: user 12.3 ms, sys: 927 µs, total: 13.2 ms
Wall time: 3.18 ms


In [12]:
# Compute pairwise labeling rates
mask_2 = torch.ones((M,M,K,K)).byte()
for i in range(M):
    mask_2[i,i,:,:] = 0

O_2 = np.einsum('aby,cdy,y->acbd', alpha, alpha, p_Y)
O_2 = torch.from_numpy(O_2).float()
O_2[1-mask_2] = 0

In [13]:
# Compute observed labeling rates
O_l = torch.from_numpy(np.einsum('aby,y->ab', alpha, p_Y)).float()

### Step 4: Try to recover $\alpha$ given $P(Y=y)$

In [14]:
def get_loss(A, P, O, O_l, O_2):
    
    # Main constraint: match empirical three-way overlaps matrix (entries O_ijk for i != j != k)
    loss_1 = torch.norm((O - torch.einsum('aby,cdy,efy,y->acebdf', [A,A,A,P]))[mask])**2
    
    # Col-wise stochastic: \sum_y' P(\lf=y'|Y=y) = 1.0
    loss_2 = torch.norm(torch.sum(A, 1) - 1)**2
    
    # Row-wise constraint: match observed labeling rates P(\lf=y') = \sum_y P(Y=y) P(\lf=y'|Y=y)
    loss_3 = torch.norm(O_l - torch.einsum('aby,y->ab', [A,P]))**2
    
    # Pairwise observed: match empirical pairwise overlaps matrix (entries O_ij for i != j)
    # loss_4 = torch.norm((O_2 - torch.einsum('aby,cdy,y->acbd', [A,A,P]))[mask_2])**2
    
    return loss_1 + loss_2 + loss_3 # + loss_4

def train_model(A, P, O, O_l, O_2, n_epochs=10, lr=0.01, momentum=0, print_every=1):
    optimizer = optim.SGD([A], lr=lr, momentum=momentum)
    
    for epoch in range(n_epochs):
        epoch_loss = 0.0
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass to calculate outputs
        loss = get_loss(A, P, O, O_l, O_2)

        # Backward pass to calculate gradients
        loss.backward()

        # Perform optimizer step
        optimizer.step()

        # Keep running sum of losses
        epoch_loss += loss.detach()

        # Report progress
        if epoch % print_every == 0:
            msg = f"[E:{epoch}]\tLoss: {epoch_loss:.8f}"
            print(msg)

def train_model_lbfgs(A, P, O, O_l, O_2, n_epochs=10, lr=1, print_every=1):
    optimizer = optim.LBFGS([A], lr=lr)
    
    for epoch in range(n_epochs):        
        def closure():
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass to calculate outputs
            loss = get_loss(A, P, O, O_l, O_2)

            # Backward pass to calculate gradients
            loss.backward()

            # Report progress
            if epoch % print_every == 0:
                msg = f"[E:{epoch}]\tLoss: {loss.detach():.8f}"
                print(msg)
            
            return loss

        # Perform optimizer step
        optimizer.step(closure)

In [15]:
A = nn.Parameter(torch.from_numpy(np.random.rand(M, K, K)).float()).float()
P = torch.from_numpy(p_Y).float()

# train_model(A, P, O, O_l, O_2, n_epochs=10000, lr=0.005, momentum=0.9, print_every=1000)
train_model_lbfgs(A, P, O, O_l, O_2, n_epochs=10, print_every=1)

print(f"Param estimation error: {np.mean(np.abs(A.detach().numpy() - alpha))}")

[E:0]	Loss: 425.36105347
[E:0]	Loss: 340.80267334
[E:0]	Loss: 109.66790771
[E:0]	Loss: 53.99650574
[E:0]	Loss: 26.19582939
[E:0]	Loss: 14.41246223
[E:0]	Loss: 6.41153574
[E:0]	Loss: 2.53658915
[E:0]	Loss: 1.73112893
[E:0]	Loss: 1.07614517
[E:0]	Loss: 0.84402746
[E:0]	Loss: 0.68905640
[E:0]	Loss: 0.50893801
[E:0]	Loss: 0.35955188
[E:0]	Loss: 0.22685249
[E:0]	Loss: 0.20317608
[E:0]	Loss: 0.19488670
[E:0]	Loss: 0.18258806
[E:0]	Loss: 0.16458936
[E:0]	Loss: 0.16071397
[E:1]	Loss: 0.14896670
[E:1]	Loss: 0.14194909
[E:1]	Loss: 0.12217625
[E:1]	Loss: 0.09708909
[E:1]	Loss: 0.06923153
[E:1]	Loss: 0.03416957
[E:1]	Loss: 0.03278351
[E:1]	Loss: 0.02825810
[E:1]	Loss: 0.02700087
[E:1]	Loss: 0.02570152
[E:1]	Loss: 0.02235990
[E:1]	Loss: 0.02113901
[E:1]	Loss: 0.01847202
[E:1]	Loss: 0.01502717
[E:1]	Loss: 0.01050317
[E:1]	Loss: 0.00834673
[E:1]	Loss: 0.00520078
[E:1]	Loss: 0.00453616
[E:1]	Loss: 0.00375597
[E:1]	Loss: 0.00274836
[E:2]	Loss: 0.00172170
[E:2]	Loss: 0.00121723
[E:2]	Loss: 0.00072347
[E

In [16]:
A[0]

tensor([[0.4755, 0.2726, 0.2694],
        [0.0605, 0.1885, 0.2899],
        [0.4640, 0.5388, 0.4407]], grad_fn=<SelectBackward>)

In [17]:
alpha[0]

array([[0.4754549 , 0.27264056, 0.26943463],
       [0.06054756, 0.18850857, 0.28989351],
       [0.46399755, 0.53885088, 0.44067186]])

In [18]:
A[0].sum(1)

tensor([1.0175, 0.5390, 1.4435], grad_fn=<SumBackward1>)

In [19]:
alpha[0].sum(1)

array([1.01753008, 0.53894963, 1.44352029])